In [ ]:
from typing import TypedDict, List, Annotated
from langgraph.graph import StateGraph, START, END, MessagesState
from langgraph.graph.message import add_messages
from langchain_ollama import ChatOllama
from IPython.display import Image, display
from langchain_core.messages import (
    HumanMessage,
    SystemMessage,
    RemoveMessage,
    trim_messages,
)
from langchain_deepseek import ChatDeepSeek
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
import os
from pydantic import BaseModel, Field
from rich.markdown import Markdown
from rich.console import Console
from enum import Enum
import pandas as pd
import scipy
from pprint import pprint
from graphviz import Source
from langgraph.checkpoint.memory import MemorySaver
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.store.memory import InMemoryStore

# Load Data


In [4]:
df = pd.read_csv("data3.csv")

In [5]:
streams = list(df["Stream"].unique())
student_names = list(df["Student Name"].unique())
subject_categories = list(df["Subject Category"].unique())
subjects = list(df["Subject"].unique())
subject_strands = list(df["Subject Strand"].unique())
terms = list(df["Term"].unique())
grades = list(df["Grade"].unique())


subject_strand_to_subject = {
    "English Reading": "English",
    "English Writing": "English",
    "Vocabulary": "Spanish",
    "Spanish Grammar": "Spanish",
    "Algebra": "Math",
    "Geometry": "Math",
    "Mechanics": "Physics",
    "Thermodynamics": "Physics",
    "Physical Geography": "Geography",
    "Human Geography": "Geography",
    "Ancient History": "History",
    "Medieval History": "History",
}

In [6]:
data = []

for stream in streams:
    # Get specific stream
    stream_df = df[df["Stream"] == stream]

    # Get details of this stream
    student_names = list(stream_df["Student Name"].unique())
    subject_strands = list(stream_df["Subject Strand"].unique())

    # For each student look at how well they did in each subject
    for student_name in student_names:
        student_df = stream_df[stream_df["Student Name"] == student_name]

        for subject_strand in subject_strands:
            subject = subject_strand_to_subject[subject_strand]
            # Get Mean and Median for strean in specific strand

            stream_subject_strand_df = stream_df[
                stream_df["Subject Strand"] == subject_strand
            ]

            stream_subject_strand_mean = (
                stream_subject_strand_df["Grade Numeric"].mean().round(1)
            )

            # stream_subject_strand_median = stream_subject_strand_df[
            #     "Grade Numeric"
            # ].median()

            stream_subject_strand_std = stream_subject_strand_df["Grade Numeric"].std()

            # Get stream mean in a particular strand
            student_subject_strand_df = student_df[
                student_df["Subject Strand"] == subject_strand
            ]

            student_subject_strand_mean = (
                student_subject_strand_df["Grade Numeric"].mean().round(1)
            )

            # student_z_score = (
            #     (student_subject_strand_mean - stream_subject_strand_mean)
            #     / stream_subject_strand_std
            #     # if stream_subject_strand_std != 0
            #     # else 0
            # )

            student_percentile = scipy.stats.percentileofscore(
                stream_subject_strand_df["Grade Numeric"],
                student_subject_strand_mean,
            ).round(1)

            data.append(
                [
                    stream,
                    student_name,
                    subject,
                    subject_strand,
                    student_subject_strand_mean,
                    student_percentile,
                ]
            )


subject_strand_percentile_df = pd.DataFrame(
    data,
    columns=[
        "Stream",
        "Student Name",
        "Subject",
        "Subject Strand",
        "Grade Numeric",
        "Percentile",
    ],
)

display(subject_strand_percentile_df.head(24))
display(subject_strand_percentile_df.tail(24))

,Stream,Student Name,Subject,Subject Strand,Grade Numeric,Percentile
0,Yellow,Holly,English,English Reading,4.3,77.8
1,Yellow,Holly,English,English Writing,4.3,87.0
2,Yellow,Holly,Spanish,Vocabulary,4.7,87.0
3,Yellow,Holly,Spanish,Spanish Grammar,4.3,66.7
4,Yellow,Holly,Math,Algebra,4.7,81.5
5,Yellow,Holly,Math,Geometry,4.7,77.8
6,Yellow,Holly,Physics,Mechanics,4.3,77.8
7,Yellow,Holly,Physics,Thermodynamics,3.7,51.9
8,Yellow,Holly,Geography,Physical Geography,4.0,76.9
9,Yellow,Holly,Geography,Human Geography,5.0,86.1


,Stream,Student Name,Subject,Subject Strand,Grade Numeric,Percentile
396,Blue,Mallory,English,English Reading,1.3,33.3
397,Blue,Mallory,English,English Writing,1.7,20.4
398,Blue,Mallory,Spanish,Vocabulary,3.3,75.9
399,Blue,Mallory,Spanish,Spanish Grammar,1.7,33.3
400,Blue,Mallory,Math,Algebra,2.3,55.6
401,Blue,Mallory,Math,Geometry,4.0,84.3
402,Blue,Mallory,Physics,Mechanics,2.7,46.3
403,Blue,Mallory,Physics,Thermodynamics,2.3,48.1
404,Blue,Mallory,Geography,Physical Geography,4.3,87.0
405,Blue,Mallory,Geography,Human Geography,2.3,55.6


# Load LLM


In [7]:
# ollama_llm_model = "llama3.2:1b"

# llm = ChatOllama(model=ollama_llm_model, temperature=1)

load_dotenv()

deepseek_model = "deepseek-chat"
DEEPSEEK_API_KEY = os.getenv("DEEPSEEK_API_KEY")
os.environ["DEEPSEEK_API_KEY"] = DEEPSEEK_API_KEY

llm = ChatDeepSeek(model=deepseek_model)

# Agent


In [ ]:
# Output Formats
class ClassificationEnum(Enum):
    grade_probability = "Grade Probability"
    strength_or_weakness = "Strength and weakness of student"
    career_recommendations = "Career recommendation for student"
    other = "other"


class Classification(BaseModel):
    classification: ClassificationEnum = Field("Classificaton of the query")


class Recommendation(BaseModel):
    stream: str = Field(description="The class the student is in")
    name: str = Field(description="The student's name")
    subject: str = Field(description="The subject")
    subject_strand: str = Field(description="The specific topic within the subject")
    grade_numeric: float = Field(description="The grade bound between 1 and 5")
    percentile: float = Field(description="Percentile of student")
    strength_or_weakness: str = Field(
        description="Describe if the student is strong or weak relative to class, use percentile"
    )
    recommendations: str = Field(
        description="A generic recommendation on what they should do. Maybe focus on another strand within the subject or focus on another subject altogether"
    )


class RecommendationList(BaseModel):
    recommendations: List[Recommendation]


# State
class State(MessagesState):
    summary: str

    # text: str
    # classification: ClassificationEnum
    # recommendations: List[Recommendation]


# Initialize Graph Builder
graph_builder = StateGraph(State)


def summarisation_node(state: State):
    summary = state.get("summary", "")

    if summary:
        summary_message = (
            f"This is a summary of the conversation to date: {summary}\n\n"
            "Extend the summary by taking into account the new messages above:"
        )

    else:
        summary_message = "Create a summary of the conversation above:"

    # Add previous messages togther with summary message
    messages = state["messages"] + [HumanMessage(content=summary_message)]
    response = llm.invoke(messages)

    # Delete all but the 2 most recent messages
    delete_messages = [RemoveMessage(id=m.id) for m in state["messages"][:-2]]
    return {"summary": response.content, "messages": delete_messages}


# def classification_node(state: State):
#     prompt = PromptTemplate(
#         input_variables=["text"],
#         template="""
#     Classify the folowing text into one of the categories:
#     1. Grade Probability
#     2. Strength and weakness of student
#     3. Career recommendation for student
#     4. Other

#     Text: {text}

#     Category:
#     """,
#     )

#     message = HumanMessage(content=prompt.format(text=state["text"]))

#     llm_with_structured_output = llm.with_structured_output(Classification)

#     # classification = llm.invoke([message]).content.strip()

#     classification: Classification = llm_with_structured_output.invoke([message])

#     return {"classification": classification.classification.value}


# Conditional Edges
def should_summarise(state: State):
    max_messages = 10
    if len(state["messages"]) > max_messages:
        return "summarisation_node"
    else:
        return END


# Define Nodes
def chatbot(state: State):
    messages = [
        SystemMessage(
            content=f"""
            You are an AI assistant helping me categorize the strengths and weaknesses of each student relative to their stream.
            
            The grade numeric is from 1 to 5.

            The percentile is there too.

            In case a student name who does not exist is input, return a general response.
            
            Focus on giving your response per student basis.

            Here is the tabular data:
            
            {subject_strand_percentile_df.to_string()}
            """
        )
    ]

    # llm_with_structured_output = llm.with_structured_output(RecommendationList)

    # recommendations: RecommendationList = llm_with_structured_output.invoke(messages)
    messages = [llm.invoke(state["messages"])]

    # return {"recommendations": recommendations, "messages": messages}
    return {"messages": messages}


# Add Nodes
graph_builder.add_node("chatbot", chatbot)
# graph_builder.add_node("classification_node", classification_node)
graph_builder.add_node("summarisation_node", summarisation_node)
# Add Edges
# graph_builder.add_edge(START, "classification_node")
# graph_builder.fadd_edge("classification_node", "chatbot")
graph_builder.add_edge(START, "chatbot")
graph_builder.add_conditional_edges("chatbot", should_summarise)
graph_builder.add_edge("summarisation_node", END)

# Memory
checkpointer = SqliteSaver.from_conn_string("checkpoints.sqlite")
in_memory_store = InMemoryStore()

# Compile Graph
graph = graph_builder.compile(checkpointer=checkpointer, store=in_memory_store)

# # Display Graph
# try:
#     display(Image(graph.get_graph().draw_mermaid_png()))
# except Exception as e:
#     print(f"Error when displaying graph: {e}")
#     pass

graph_source = graph.get_graph().draw_mermaid()
print(graph_source)
# source = Source(graph_source)
# source.render('graph_output', format='png', cleanup=True)  # Or any other format you need

---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	chatbot(chatbot)
	summarisation_node(summarisation_node)
	__end__([<p>__end__</p>]):::last
	__start__ --> chatbot;
	summarisation_node --> __end__;
	chatbot -.-> summarisation_node;
	chatbot -.-> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



In [ ]:
config = {"configurable": {"thread_id": "1"}}


def stream_graph_updates(user_input: str):
    graph_input = {"messages": [HumanMessage(user_input)], "text": user_input}
    for event in graph.stream(input=graph_input, config=config):
        for key, value in event.items():
            if key == "classification_node":
                console = Console()
                print(key)
                print("-" * 50)
                # md = Markdown(value["classification"])
                # console.print(md)
                print(value)
                print("=" * 50)
            else:
                print(key)
                print("-" * 50)
                print(value)
                print("=" * 50)


def invoke_full_graph(user_input: str) -> State:
    # Create the initial input state
    graph_input = {
        "messages": [
            SystemMessage(
                content=f"""
            You are an AI assistant helping me categorize the strengths and weaknesses of each student relative to their stream.
            
            The grade numeric is from 1 to 5.

            The percentile is there too.

            In case a student name who does not exist is input, return a general response.
            
            Focus on giving your response per student basis.

            Here is the tabular data:
            
            {subject_strand_percentile_df.to_string()}            
            """
            ),
            HumanMessage(user_input),
        ],
        "text": user_input,
    }

    # Process the full graph at once
    result = graph.invoke(
        input=graph_input, config=config
    )  # This will invoke all nodes and return the result

    # # Output the result
    # print("Final result after running the entire graph:")
    # for key, value in result.items():
    #     print(key)
    #     print("-" * 50)
    #     pprint(value)
    #     print("=" * 50)

    return result


result = invoke_full_graph(
    f"Get output for {subject_strand_percentile_df.iloc[0]['Student Name']}"
)

# while True:
#     try:
#         user_input = input("User: ")
#         if user_input.lower() in ["quit", "exit", "q"]:
#             print("Goodbye!")
#             break

#         # Invoke the full graph for each user input
#         invoke_full_graph(user_input)

#     except Exception as e:
#         print(f"Error occurred: {e}")
#         break


# while True:
#     try:
#         user_input = input("User: ")
#         if user_input.lower() in ["quit", "exit", "q"]:
#             print("Goodbye!")
#             break

#         stream_graph_updates(user_input)
#     except Exception as e:
#         # fallback if input() is not available
#         # user_input = "What do you know about LangGraph?"
#         # print("User: " + user_input)
#         # stream_graph_updates(user_input)
#         print(e)
#         break

In [25]:
# for key, value in result.items():
#     print(key)
#     print("-" * 50)
#     pprint(value)
#     print("=" * 50)

for item in result["messages"]:
    pprint(item.pretty_print())

================================ System Message ================================


            You are an AI assistant helping me categorize the strengths and weaknesses of each student relative to their stream.
            
            The grade numeric is from 1 to 5.

            The percentile is there too.

            In case a student name who does not exist is input, return a general response.
            
            Focus on giving your response per student basis.

            Here is the tabular data:
            
                 Stream Student Name    Subject      Subject Strand  Grade Numeric  Percentile
0    Yellow        Holly    English     English Reading            4.3        77.8
1    Yellow        Holly    English     English Writing            4.3        87.0
2    Yellow        Holly    Spanish          Vocabulary            4.7        87.0
3    Yellow        Holly    Spanish     Spanish Grammar            4.3        66.7
4    Yellow        Holly       Math       

# To do

1. Understand how memory works
2. Put existing memory in Langgraph memory


# To Do

1. Fine-tune suggestions
